In [ ]:
from pathlib import Path
from datasets import load_dataset
from transformers import TrainingArguments
from span_marker import SpanMarkerModel, Trainer, SpanMarkerModelCardData

Загрузка и подготовка датасета

In [ ]:
dataset_id = "DFKI-SLT/few-nerd"
dataset_name = "FewNERD"
dataset = load_dataset(dataset_id, "supervised")
dataset = dataset.remove_columns("ner_tags")
dataset = dataset.rename_column("fine_ner_tags", "ner_tags")
labels = dataset["train"].features["ner_tags"].feature.names

Инициализация SpanMarkerModel и подготовка предварительно обученной модели

In [ ]:
encoder_id = "bert-base-cased"
model_id = f"tomaarsen/span-marker-{encoder_id}-fewnerd-fine-super"
model = SpanMarkerModel.from_pretrained(
    encoder_id,
    labels=labels,
    # SpanMarker hyperparameters:
    model_max_length=256,
    marker_max_length=128,
    entity_max_length=8,
    # Model card arguments
    model_card_data=SpanMarkerModelCardData(
        model_id=model_id,
        encoder_id=encoder_id,
        dataset_name=dataset_name,
        dataset_id=dataset_id,
        license="cc-by-sa-4.0",
        language="en",
    ),
)

Подготовка к обучению

In [ ]:
output_dir = Path("../models") / model_id
args = TrainingArguments(
    output_dir=output_dir,
    report_to="wandb",
    # Training Hyperparameters:
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    warmup_ratio=0.1,
    bf16=True,  # Replace `bf16` with `fp16` if your hardware can't use bf16.
    # Other Training parameters
    logging_first_step=True,
    logging_steps=50,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=3000,
    save_total_limit=2,
    dataloader_num_workers=2,
)

Инициализация обучения и обучение

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
)
trainer.train()

Подсчет и сохранение метрик

In [ ]:
metrics = trainer.evaluate(dataset["test"], metric_key_prefix="test")
trainer.save_metrics("test", metrics)

Сохраняем модель

In [ ]:
trainer.save_model(output_dir / "checkpoint-final")